# Database Support

## APIs and usage

## TODO

## Helper defining 

In [1]:
from IMP import NotebookLoader, NotebookFinder
from peewee import SqliteDatabase, Model, CharField, TextField, ForeignKeyField, DateTimeField, FloatField
from tools import get_addr_port, get_regs_list, get_reg_names, cached_property, get_md5_string
from IPython.core.debugger import Tracer; set_trace = Tracer(colors='linux')
from datetime import datetime
from collections import OrderedDict


Docstring = '''
Database relative functional module
'''


db = SqliteDatabase('data.db')

# table baselog
class BaseLog(Model):
    tmstamp = DateTimeField()
    class Meta:
        database = db

# table config
class Config(Model):
    '''
    One config points to one log table. While insert a configure into config table,
    a related log table should be created. So in overrided function save(), there is 
    a get_related_log() to get that log table and create it.
    '''
    sn = CharField(32)
    addr_info = CharField(32)
    regs_list = CharField(28)
    reg_names = CharField(128)
    info = TextField()
    md5 = CharField(32, unique=True)    
    class Meta:
        database = db
    
    def __init__(self, **argv):
        # md5 automatic generated
        if argv:
            md5string = ''.join(argv.values())
            md5 = argv['md5'] = get_md5_string(md5string)

            # data validation
            addr, self.port = get_addr_port(argv['addr_info'])
            rl = get_regs_list(argv['regs_list'])
            rn = get_reg_names(argv['reg_names'])        
            print('All data valid')       
            
        super().__init__(**argv)  
    
    def save(self, force_insert=False, only=None):
        # when finished data validation, a related log table should be created(related on md5)
        # conventional saving process  
        super().save(force_insert, only)
        print('saved')        
        
        # create new Log Table class and create it in database
        Log = self.get_related_log()
        Log.create_table()
        
    def get_related_log(self):
        ref_dict = OrderedDict()
        rl = get_regs_list(self.regs_list)
        rn = get_reg_names(self.reg_names)
        for name, reg in zip(rn, rl):
            ref_dict['_'.join([name, str(reg)])] = FloatField()

        ref_dict.update(OrderedDict(BaseLog.__dict__))
        ref_dict['__module__'] = getattr(self, '__module__', None)
        
        LogTable = type(self.md5, (BaseLog,), ref_dict)
        return LogTable
        
        
class BaseLog(Model):            
    owner = ForeignKeyField(Config)
    tmstamp = DateTimeField()
    class Meta:
        database = db
    
db.create_table(Config)

x = Config.create(sn='slv030912041021', addr_info='192.168.1.10@202',\
                  regs_list='20-23,50', reg_names='ch1, ch2, ch3, ch4, ch5', info='info: testing')


importing Jupyter notebook from tools.ipynb
All data valid
saved


### **For peewee test**

In [1]:
from peewee import *

db1 = SqliteDatabase('people.db')

class Person(Model):
    name = CharField()
    birthday = DateField()
    is_relative = BooleanField()

    class Meta:
        database = db1 # This model uses the "people.db" database.

class Pet(Model):
    owner = ForeignKeyField(Person, related_name='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db1 # this model uses the "people.db" database
        
db1.create_tables([Person, Pet])

from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
uncle_bob.save() # bob is now stored in the database

grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1), is_relative=True)
herb = Person.create(name='Herb', birthday=date(1950, 5, 5), is_relative=False)
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

### **These code below have been deprecated**

```python
from IPython.core.debugger import Tracer; set_trace = Tracer(colors='linux')

Docstring = '''
Database relative functional module
'''

import tools as tl
import sqlite3
from tools import cached_property


def sqlexec(db, *sql_val):
    cur = db.cursor()
    cur.execute(*sql_val)
    cur.close()
    

class ConfigDB(object):
    @cached_property
    def db(self):
        print('connecting config.db')
        return sqlite3.connect('config.db')
    
    @cached_property
    def cur(self):
        print('cursor been got')
        return self.db.cursor()
    
    def __init__(self, func):
        self.func = func
        self.__dict__['__doc__'] = func.__doc__
    
    def __call__(self, **argv):
        if 'verb' in argv:
            verb = argv['verb']
            not_closed = 'db' in self.__dict__ and 'cur' in self.__dict__
            if verb == 'commit':
                self.db.commit()
            elif verb == 'close' and not_closed: 
                self.db.commit()
                self.cur.close()
                self.db.close()
                del self.cur
                del self.db
            return
        
        sql = self.func(**argv)
        cur = self.cur
        excpt = False
        if isinstance(sql, str):
            cur.execute(sql)
        elif isinstance(sql, tuple) and len(sql) == 2:
            cur.execute(*sql)
        else:
            raise(ValueError('sql is not string or (string, values): %s'%sql))
        return sql
    
    def __del__(self):
        print('closing')
        self.__call__(verb = 'close')
    
@ConfigDB    
def add_config(sn, addr_info, regs_list, reg_names, info):
    '''
    def add_config(sn, addr_info, regs_list, reg_names, info):
        ...
        
    sn: serial number of xstream
    addr_info: ip@port or address@port
    regs_list: list of registers, from small to large, split by comma(","), eg: 10-20,25,30-32
    reg_names: names of registers, like CO2, H2, Temp, CH4, ch_1, ch_2... numbers of names in reg_names and registers in reg_list MUST BE matched
    info: some description, caption, infomation
    '''    
    
    args_dict = locals()
    addr, port = tl.get_addr_port(addr_info)
    regs = tl.get_regs_list(regs_list)
    names = tl.get_reg_names(reg_names)
    
    try:
        assert(len(regs) == len(names))
    except AssertionError as e:
        print('numbers of names in reg_names and registers in reg_list MUST BE matched')
        print('regs num: %d, names num: %d'% (len(regs), len(names)))
        raise(tl.PrintColorfulTraceBack)
        
    sql = ('INSERT INTO config(%s,%s,%s,%s,%s) VALUES(?,?,?,?,?)'%tuple(args_dict.keys()), list(args_dict.values()))
    return sql

@ConfigDB
def del_config(sn):
    pass
```